In [1]:
import numpy as np
import pymc3 as pm
from tqdm.notebook import tqdm
from tensorflow_privacy.privacy.optimizers.dp_optimizer import *
import pandas as pd
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import random; random.seed(42)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf

%load_ext autoreload
%autoreload 2

tf.compat.v1.enable_eager_execution()
assert tf.executing_eagerly

Using TensorFlow backend.


In [2]:
epochs = 1
batch_size = 100

DP-SGD has three privacy-specific hyperparameters and one existing hyperamater that you must tune:

1. `l2_norm_clip` (float) - The maximum Euclidean (L2) norm of each gradient that is applied to update model parameters. This hyperparameter is used to bound the optimizer's sensitivity to individual training points. 
2. `noise_multiplier` (float) - The amount of noise sampled and added to gradients during training. Generally, more noise results in better privacy (often, but not necessarily, at the expense of lower utility).
3.   `microbatches` (int) - Each batch of data is split in smaller units called microbatches. By default, each microbatch should contain a single training example. This allows us to clip gradients on a per-example basis rather than after they have been averaged across the minibatch. This in turn decreases the (negative) effect of clipping on signal found in the gradient and typically maximizes utility. However, computational overhead can be reduced by increasing the size of microbatches to include more than one training examples. The average gradient across these multiple training examples is then clipped. The total number of examples consumed in a batch, i.e., one step of gradient descent, remains the same. The number of microbatches should evenly divide the batch size. 

In [3]:
l2_norm_clip = 0.5
noise_multiplier = 1
gaussian_stdev = l2_norm_clip * noise_multiplier
gaussian_noise_var = gaussian_stdev ** 2
public_var_multiplier = 1 # Multiply public variance by this multiplier
num_microbatches = batch_size

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [4]:
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255
train_data = np.expand_dims(train_data, len(train_data.shape))
test_data = np.expand_dims(test_data, len(test_data.shape))

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

assert train_data.min() == 0.
assert train_data.max() == 1.
assert test_data.min() == 0.
assert test_data.max() == 1.

In [5]:
from sklearn.model_selection import train_test_split

# test_size refers to private data size
public_data, private_data, public_labels, private_labels = \
    train_test_split(train_data, train_labels, test_size=199/200)

num_batches = private_data.shape[0] // batch_size

In [6]:
print(public_data.shape)
print(public_labels.shape)
print(private_data.shape)
print(private_labels.shape)

(300, 28, 28, 1)
(300, 10)
(59700, 28, 28, 1)
(59700, 10)


In [7]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=private_labels.shape[0], batch_size=batch_size, noise_multiplier=noise_multiplier, epochs=epochs, delta=1e-5)

DP-SGD with sampling rate = 0.168% and noise_multiplier = 1 iterated over 597 steps satisfies differential privacy with eps = 1.07 and delta = 1e-05.
The optimal RDP order is 12.0.


(1.0661600902513837, 12.0)

In [99]:
# CNN model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# ~10k parameters, ~95% accuracy
def cnn_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(2, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=train_data.shape[1:]))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Flatten())
    model.add(Dense(8, activation='relu'))
    model.add(Dense(10, activation='softmax'))
#     model.load_weights('mnist_tiny_initial_weights.h5')
    return model

In [106]:
print([l.shape for l in public_grads])

[(300, 18), (300, 2), (300, 1024), (300, 8), (300, 80), (300, 10)]


In [117]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def construct_graphical_model(public_grads_flat, dp_grads):
    grad_values = [grad[0] for grad in dp_grads]
    num_parameters_by_layer = [l.shape[1] for l in public_grads_flat]
    weights_and_biases_by_layer = list(chunks(num_parameters_by_layer, 2))
    public_weights_and_biases_by_layer = list(chunks(num_parameters_by_layer, 2))
    weights_by_layer = [params[0] for params in weights_and_biases_by_layer]
    weights_beginning_index_by_layer = np.append([0], weights_by_layer[:-1])
    graphical_model = pm.Model()
    layers = []
    bias_idx = 0
    weight_idx = 0
    for layer_num, num_params in enumerate(num_parameters_by_layer):
        if layer_num % 2 == 1:
            for param in range(num_params):
                observed_values = public_grads_flat[layer_num][:, param]
                # Bias
                with graphical_model:
                    exec(f"bias_prior_{bias_idx} = pm.Normal('bias_prior_{bias_idx}', mu=0, sigma=1)")
                    exec(f"bias_{bias_idx} = pm.Normal('bias_{bias_idx}', \
                             mu=bias_prior_{bias_idx}, sigma={gaussian_stdev}, observed=observed_values)")
                bias_idx += 1
        else:
            # Weight
            weight_layer_num = layer_num // 2
            for param in range(num_params):
                observed_values = public_grads_flat[layer_num][:, param]
                if weight_layer_num > 0:
                    with graphical_model:
                        exec(f"weight_prior_{weight_idx} = pm.Normal('weight_prior_{weight_idx}', mu=0, sigma=1)")
                    previous_layer_weights = [f'weight_{j}' for j in 
                                              range(weights_beginning_index_by_layer[weight_layer_num-1],
                                                    weights_beginning_index_by_layer[weight_layer_num])]
                    previous_layer_weights_string = ' + '.join(previous_layer_weights)
                    with graphical_model:
                        exec(f"weight_{weight_idx} = pm.Normal('weight_{weight_idx}', \
                                 mu=weight_prior_{weight_idx} * ({previous_layer_weights_string}), sigma={gaussian_stdev}, \
                                 observed=observed_values)")
                else:
                    with graphical_model:
                        exec(f"weight_{weight_idx} = pm.Normal('weight_{weight_idx}', mu=0, sigma={gaussian_stdev}, \
                                 observed=observed_values)")
                weight_idx += 1
    return graphical_model

In [ ]:
model = construct_graphical_model(public_grads, grads)

In [ ]:
import time
start = time.time()
map_estimate = pm.find_MAP(model=model)
end = time.time()
print(end - start)

In [ ]:
end - start

In [101]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [102]:
def get_public_grads_weights_flat(public_x, public_y, loss_fn, model):
    public_grads = []
    # x needs to have extra dimension for number of examples,
    # even if it's 1 for our case
    public_x = np.expand_dims(public_x, axis=1)
    for x, y in zip(public_x, public_y):
#     for x, y in tqdm(zip(public_x, public_y), total=public_x.shape[0], desc='Public Dataset Iter'):
        with tf.GradientTape() as tape:
            loss_value = loss_fn(y, model(x))
            grad = tape.gradient(loss_value, model.trainable_weights)
        if not public_grads:
            public_grads = [[] for _ in grad]
        for i, t in enumerate(grad):
            public_grads[i].append(t.numpy().flatten())
    public_grads = [np.asarray(l) for l in public_grads]
    return public_grads

def evaluate_model(model, loss_fn, x, y):
    pred = model(x)
    loss = np.mean(loss_fn(y, pred).numpy())
    acc = np.mean(tf.keras.metrics.categorical_accuracy(y, pred).numpy())
    return (loss, acc)

In [103]:
from tensorflow.keras.callbacks import EarlyStopping

pretrained_model = cnn_model()
pretrained_model.compile(optimizer='adam',
                       loss=loss_fn, metrics=['accuracy'])
baseline_history = pretrained_model.fit(public_data, public_labels,
                    epochs=50,
                    batch_size=batch_size,
                    verbose=0,
                    callbacks=[TQDMNotebookCallback(), EarlyStopping(monitor='acc', patience=5)])
evaluate_model(pretrained_model, loss_fn, test_data, test_labels)

(1.9748842, 0.5889)

In [104]:
custom_model = cnn_model()
custom_optimizer = DPAdamGaussianOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches)
custom_model.compile(optimizer=custom_optimizer, loss=loss_fn, metrics=['accuracy'])

dpsgd_model = cnn_model()
dpsgd_optimizer = DPAdamGaussianOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches)
dpsgd_model.compile(optimizer=dpsgd_optimizer, loss=loss_fn, metrics=['accuracy'])

In [105]:
# Iterate over epochs.
custom_loss_batches = []
custom_acc_batches = []

dpsgd_loss_batches = []
dpsgd_acc_batches = []

# Used for picking a random minibatch
idx_array = np.arange(private_data.shape[0])

for epoch in tqdm(range(epochs), desc='Epoch'):

    # Iterate over the batches of the dataset.
    for step in tqdm(range(num_batches), desc='Batch'):
        
        # Pick a random minibatch
        random_idx = np.random.choice(idx_array, batch_size)
        x_batch_train = private_data[random_idx]
        y_batch_train = private_labels[random_idx]
        
        ### Normal DPSGD
    
        # Evaluate DPSGD model
        loss, acc = evaluate_model(dpsgd_model, loss_fn, test_data, test_labels)
#         print('DPSGD Loss: %.4f | Acc: %.4f' % (loss, acc))
        dpsgd_loss_batches.append(loss)
        dpsgd_acc_batches.append(acc)
    
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape(persistent=True) as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = dpsgd_model(x_batch_train)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss = lambda: loss_fn(y_batch_train, logits)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = dpsgd_optimizer.compute_gradients(loss, dpsgd_model.trainable_weights, gradient_tape=tape)

        del tape

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        dpsgd_optimizer.apply_gradients(grads)
        
        ### Our custom DPSGD
    
        # Evaluate custom model
        loss, acc = evaluate_model(custom_model, loss_fn, test_data, test_labels)
#         print('Custom Loss: %.4f | Acc: %.4f' % (loss, acc))
        custom_loss_batches.append(loss)
        custom_acc_batches.append(acc)

        public_grads = get_public_grads_weights_flat(public_data, public_labels, loss_fn, custom_model)
        
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape(persistent=True) as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = custom_model(x_batch_train)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss = lambda: loss_fn(y_batch_train, logits)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = custom_optimizer.compute_gradients(loss, custom_model.trainable_weights,
                                                              gradient_tape=tape)

        del tape
        

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        custom_optimizer.apply_gradients(grads)

KeyboardInterrupt: 

In [ ]:
evaluate_model(custom_model, loss_fn, test_data, test_labels)

In [ ]:
evaluate_model(dpsgd_model, loss_fn, test_data, test_labels)

In [ ]:
baseline_loss_per_batch = [l for e in baseline_history.history['val_loss'] for l in e ]
baseline_loss_per_epoch = [np.mean(e) for e in baseline_history.history['val_loss']]
baseline_acc_per_epoch = baseline_history.history['val_acc']

In [ ]:
metrics = pd.DataFrame({
#                         'custom_loss': custom_loss_batches,
#                         'dpsgd_loss': dpsgd_loss_batches,
                        'custom_acc': custom_acc_batches,
                        'dpsgd_acc': dpsgd_acc_batches,
#                         'grad_norm': public_norms,
#                         'baseline_loss': baseline_loss_per_epoch,
#                         'baseline_acc': baseline_acc_per_epoch
                       })
print(metrics)

In [ ]:
ax = sns.lineplot(data=metrics[:200])
ax.set(xlabel='Minibatch', ylabel='Loss/Acc', 
       title='MNIST Per Weight Bayesian (Norm Clip={}, Public Size={})'.format(l2_norm_clip, public_data.shape[0]))
plt.savefig('mnist_per-weight-bayesian-pretrained_variance{}_dpsgd-norm{}.png'
            .format(public_var_multiplier, l2_norm_clip))